# Single-Cell RNA-Seq Analysis Workflow with Scanpy

**Author:** scRNA-Seq Analysis Team  
**Dataset:** 3k PBMCs from 10x Genomics  
**Goal:** Perform a complete scRNA-Seq analysis pipeline from raw counts to annotated cell types

---

## Overview

This notebook demonstrates a standard single-cell RNA-sequencing (scRNA-Seq) analysis workflow using the Python package `scanpy`. The workflow includes:

1. **Data Loading**: Import 10x Genomics formatted data
2. **Quality Control**: Filter low-quality cells and genes
3. **Normalization**: Normalize and log-transform expression data
4. **Feature Selection**: Identify highly variable genes
5. **Dimensionality Reduction**: PCA and UMAP
6. **Clustering**: Leiden algorithm for cell clustering
7. **Marker Gene Identification**: Find cluster-specific genes
8. **Cell Type Annotation**: Assign biological identities to clusters

---

## 1. Setup and Configuration

Import required libraries and configure analysis parameters.

In [ ]:
# Import libraries
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure scanpy settings
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white', frameon=False)

In [ ]:
# Define paths
DATA_DIR = Path('../data/pbmc3k/filtered_gene_bc_matrices/hg19/')
FIGURES_DIR = Path('../figures/')
FIGURES_DIR.mkdir(exist_ok=True)

# Configure output directory for scanpy figures
sc.settings.figdir = FIGURES_DIR

## 2. Data Loading

Load the 10x Genomics PBMC dataset. The dataset contains gene expression measurements for approximately 3,000 peripheral blood mononuclear cells (PBMCs).

In [ ]:
# Load the data using scanpy's read_10x_mtx function
# This function reads the matrix.mtx, genes.tsv, and barcodes.tsv files
adata = sc.read_10x_mtx(
    DATA_DIR,
    var_names='gene_symbols',  # Use gene symbols as variable names
    cache=True  # Cache the data for faster subsequent loading
)

print(f"\nDataset shape: {adata.shape}")
print(f"Number of cells: {adata.n_obs}")
print(f"Number of genes: {adata.n_vars}")

In [ ]:
# Display basic information about the dataset
adata

## 3. Quality Control (QC)

Quality control is crucial for ensuring robust downstream analysis. We will:

- Calculate QC metrics (number of genes per cell, total counts, mitochondrial gene percentage)
- Visualize QC metrics
- Filter cells and genes based on quality thresholds

### 3.1 Calculate QC Metrics

In [ ]:
# Identify mitochondrial genes (genes starting with 'MT-')
adata.var['mt'] = adata.var_names.str.startswith('MT-')

# Calculate QC metrics
# - n_genes_by_counts: number of genes expressed per cell
# - total_counts: total counts per cell
# - pct_counts_mt: percentage of counts from mitochondrial genes
sc.pp.calculate_qc_metrics(
    adata,
    qc_vars=['mt'],
    percent_top=None,
    log1p=False,
    inplace=True
)

# Display QC metrics summary
print("\nQC Metrics Summary:")
print(adata.obs[['n_genes_by_counts', 'total_counts', 'pct_counts_mt']].describe())

### 3.2 Visualize QC Metrics

In [ ]:
# Create violin plots for QC metrics
sc.pl.violin(
    adata,
    ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
    jitter=0.4,
    multi_panel=True,
    save='_qc_violin.png'
)

In [ ]:
# Create scatter plots to visualize relationships between QC metrics
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt', save='_qc_scatter_mt.png')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', save='_qc_scatter_genes.png')

### 3.3 Filter Cells and Genes

**Filtering criteria:**
- **Cells:** Retain cells with 200-2500 genes and <5% mitochondrial counts
- **Genes:** Retain genes expressed in at least 3 cells

High mitochondrial percentage often indicates dying/dead cells.

In [ ]:
# Store original counts for later reference
print(f"Cells before filtering: {adata.n_obs}")
print(f"Genes before filtering: {adata.n_vars}")

# Filter cells based on QC metrics
sc.pp.filter_cells(adata, min_genes=200)  # Minimum 200 genes per cell
sc.pp.filter_genes(adata, min_cells=3)    # Genes expressed in at least 3 cells

# Filter cells with high mitochondrial content and too many genes
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]

print(f"\nCells after filtering: {adata.n_obs}")
print(f"Genes after filtering: {adata.n_vars}")
print(f"Cells removed: {adata.n_obs}")

## 4. Normalization and Preprocessing

Normalize the data to account for differences in library size across cells and apply log transformation.

In [ ]:
# Save raw counts for later use
adata.layers['counts'] = adata.X.copy()

# Normalize total counts per cell to 10,000 (target_sum=1e4)
# This makes counts comparable across cells
sc.pp.normalize_total(adata, target_sum=1e4)

# Log-transform the data: X = log(X + 1)
sc.pp.log1p(adata)

# Store normalized and log-transformed data
adata.raw = adata

print("Normalization and log-transformation completed.")

## 5. Feature Selection: Highly Variable Genes

Identify genes that show high variability across cells. These genes are most informative for distinguishing cell types.

In [ ]:
# Identify highly variable genes
# We select the top 2000 genes with highest variability
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=2000,
    flavor='seurat',
    subset=False  # Keep all genes but mark HVGs
)

print(f"\nNumber of highly variable genes: {sum(adata.var.highly_variable)}")

In [ ]:
# Visualize highly variable genes
sc.pl.highly_variable_genes(adata, save='_hvg.png')

In [ ]:
# Subset to highly variable genes for downstream analysis
adata = adata[:, adata.var.highly_variable]
print(f"Dataset shape after HVG selection: {adata.shape}")

## 6. Scaling and Regression

Scale the data to unit variance and zero mean. Optionally regress out unwanted sources of variation.

In [ ]:
# Regress out effects of total counts and mitochondrial percentage
# This reduces technical variation
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

# Scale data to unit variance and zero mean
# Clip values at max value of 10 to avoid excessive influence of outliers
sc.pp.scale(adata, max_value=10)

print("Scaling and regression completed.")

## 7. Dimensionality Reduction: PCA

Principal Component Analysis (PCA) reduces the dimensionality of the data while preserving most of the variance.

In [ ]:
# Compute PCA
# We compute 50 principal components
sc.tl.pca(adata, svd_solver='arpack', n_comps=50)

print("PCA computation completed.")

In [ ]:
# Visualize variance ratio explained by each PC
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50, save='_variance_ratio.png')

In [ ]:
# Visualize cells in PCA space
sc.pl.pca(adata, color='CST3', save='_pca_CST3.png')

## 8. Neighborhood Graph and UMAP

Compute a neighborhood graph of cells based on PCA coordinates, then compute UMAP for 2D visualization.

In [ ]:
# Compute neighborhood graph
# This graph will be used for clustering and UMAP
sc.pp.neighbors(
    adata,
    n_neighbors=10,
    n_pcs=40  # Use first 40 PCs
)

print("Neighborhood graph computed.")

In [ ]:
# Compute UMAP embedding
# UMAP provides a 2D representation that preserves local and global structure
sc.tl.umap(adata)

print("UMAP embedding computed.")

In [ ]:
# Visualize UMAP (without clustering yet)
sc.pl.umap(adata, save='_initial.png')

## 9. Clustering with Leiden Algorithm

The Leiden algorithm is an improved version of the Louvain algorithm for detecting communities in networks.

In [ ]:
# Perform Leiden clustering
# Resolution parameter controls the coarseness of the clustering
# Higher values lead to more clusters
sc.tl.leiden(
    adata,
    resolution=0.9,
    random_state=42
)

print(f"\nNumber of clusters identified: {len(adata.obs['leiden'].unique())}")
print("\nCluster sizes:")
print(adata.obs['leiden'].value_counts().sort_index())

In [ ]:
# Visualize clustering results on UMAP
sc.pl.umap(
    adata,
    color=['leiden'],
    legend_loc='on data',
    title='UMAP by Leiden Cluster',
    save='_leiden.png'
)

## 10. Marker Gene Identification

Identify genes that are differentially expressed in each cluster compared to all other cells.

In [ ]:
# Find marker genes for each cluster
# Uses Wilcoxon rank-sum test by default
sc.tl.rank_genes_groups(
    adata,
    groupby='leiden',
    method='wilcoxon',
    key_added='rank_genes_leiden'
)

print("Marker gene analysis completed.")

In [ ]:
# Visualize top marker genes for each cluster
sc.pl.rank_genes_groups(
    adata,
    n_genes=20,
    sharey=False,
    key='rank_genes_leiden',
    save='_marker_genes.png'
)

In [ ]:
# Display top 5 marker genes for each cluster
pd.DataFrame(adata.uns['rank_genes_leiden']['names']).head(5)

In [ ]:
# Create a heatmap of top marker genes
sc.pl.rank_genes_groups_heatmap(
    adata,
    n_genes=10,
    key='rank_genes_leiden',
    groupby='leiden',
    show_gene_labels=True,
    save='_marker_heatmap.png'
)

In [ ]:
# Dotplot of marker genes
sc.pl.rank_genes_groups_dotplot(
    adata,
    n_genes=5,
    key='rank_genes_leiden',
    groupby='leiden',
    save='_marker_dotplot.png'
)

## 11. Cell Type Annotation

Annotate clusters based on known marker genes for PBMC cell types.

**Known PBMC markers:**
- **CD4 T cells:** IL7R, CD4
- **CD8 T cells:** CD8A, CD8B
- **B cells:** CD79A, MS4A1 (CD20)
- **NK cells:** GNLY, NKG7
- **Monocytes:** CD14, LYZ
- **FCGR3A+ Monocytes:** FCGR3A, MS4A7
- **Dendritic cells:** FCER1A, CST3
- **Megakaryocytes:** PPBP (platelets)

In [ ]:
# Define canonical marker genes for PBMC cell types
marker_genes = {
    'CD4 T cells': ['IL7R', 'CD4'],
    'CD8 T cells': ['CD8A', 'CD8B'],
    'B cells': ['CD79A', 'MS4A1'],
    'NK cells': ['GNLY', 'NKG7'],
    'CD14+ Monocytes': ['CD14', 'LYZ'],
    'FCGR3A+ Monocytes': ['FCGR3A', 'MS4A7'],
    'Dendritic cells': ['FCER1A', 'CST3'],
    'Megakaryocytes': ['PPBP']
}

# Flatten the marker list for visualization
marker_genes_list = [gene for genes in marker_genes.values() for gene in genes]

# Remove duplicates while preserving order
marker_genes_list = list(dict.fromkeys(marker_genes_list))

print("Marker genes for annotation:")
print(marker_genes_list)

In [ ]:
# Visualize known marker genes on UMAP
sc.pl.umap(
    adata,
    color=marker_genes_list,
    use_raw=True,
    ncols=3,
    save='_marker_expression.png'
)

In [ ]:
# Create dotplot of marker genes by cluster
sc.pl.dotplot(
    adata,
    marker_genes_list,
    groupby='leiden',
    use_raw=True,
    dendrogram=True,
    save='_cell_type_markers.png'
)

In [ ]:
# Create heatmap of marker genes
sc.pl.heatmap(
    adata,
    marker_genes_list,
    groupby='leiden',
    use_raw=True,
    cmap='viridis',
    dendrogram=True,
    save='_cell_type_heatmap.png'
)

### Manual Annotation Based on Marker Expression

In [ ]:
# Based on the marker gene expression patterns, manually annotate clusters
# This mapping should be adjusted based on your actual data
# The numbers below are examples and may differ in your analysis

cluster_annotations = {
    '0': 'CD4 T cells',
    '1': 'CD14+ Monocytes',
    '2': 'B cells',
    '3': 'CD4 T cells',
    '4': 'CD8 T cells',
    '5': 'NK cells',
    '6': 'FCGR3A+ Monocytes',
    '7': 'Dendritic cells',
    '8': 'Megakaryocytes'
}

# Add cell type annotations to the data
adata.obs['cell_type'] = adata.obs['leiden'].map(cluster_annotations).astype('category')

print("\nCell type distribution:")
print(adata.obs['cell_type'].value_counts())

## 12. Final Visualizations

In [ ]:
# Create side-by-side UMAP plots: clusters and cell types
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Leiden clusters
sc.pl.umap(
    adata,
    color='leiden',
    legend_loc='on data',
    title='UMAP by Leiden Cluster',
    ax=axes[0],
    show=False
)

# Plot 2: Annotated cell types
sc.pl.umap(
    adata,
    color='cell_type',
    legend_loc='right margin',
    title='UMAP by Cell Type',
    ax=axes[1],
    show=False
)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'umap_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("Comparison plot saved.")

In [ ]:
# Create final annotated UMAP
sc.pl.umap(
    adata,
    color='cell_type',
    legend_loc='right margin',
    title='Annotated PBMC Cell Types',
    frameon=False,
    save='_annotated.png'
)

In [ ]:
# Visualize specific marker genes overlaid on cell types
sc.pl.umap(
    adata,
    color=['cell_type', 'CD14', 'CD3D', 'MS4A1', 'NKG7'],
    use_raw=True,
    ncols=2,
    save='_final_markers.png'
)

## 13. Save Results

In [ ]:
# Save the annotated AnnData object for future use
results_file = Path('../data/pbmc3k_processed.h5ad')
adata.write(results_file)
print(f"\nProcessed data saved to: {results_file}")

In [ ]:
# Export cell type annotations to CSV
annotations_df = adata.obs[['leiden', 'cell_type']].copy()
annotations_df.to_csv(FIGURES_DIR / 'cell_annotations.csv')
print(f"Cell annotations exported to: {FIGURES_DIR / 'cell_annotations.csv'}")

In [ ]:
# Export marker genes to CSV
marker_genes_df = pd.DataFrame(adata.uns['rank_genes_leiden']['names'])
marker_genes_df.to_csv(FIGURES_DIR / 'marker_genes.csv', index=False)
print(f"Marker genes exported to: {FIGURES_DIR / 'marker_genes.csv'}")

## 14. Summary Statistics

In [ ]:
# Print summary of the analysis
print("="*60)
print("ANALYSIS SUMMARY")
print("="*60)
print(f"\nTotal cells analyzed: {adata.n_obs}")
print(f"Total genes: {adata.n_vars}")
print(f"\nNumber of clusters: {len(adata.obs['leiden'].unique())}")
print(f"Number of cell types: {len(adata.obs['cell_type'].unique())}")
print("\nCell type distribution:")
print(adata.obs['cell_type'].value_counts())
print("\n" + "="*60)

---

## Conclusion

This workflow demonstrated a complete single-cell RNA-Seq analysis pipeline:

1. ✅ Loaded and explored 10x Genomics PBMC data
2. ✅ Performed quality control and filtering
3. ✅ Normalized and preprocessed the data
4. ✅ Selected highly variable genes
5. ✅ Reduced dimensionality with PCA and UMAP
6. ✅ Identified cell clusters using Leiden algorithm
7. ✅ Found cluster-specific marker genes
8. ✅ Annotated biological cell types
9. ✅ Generated publication-quality visualizations

**Key outputs:**
- UMAP visualizations showing cluster structure and cell types
- Identified marker genes for each cell population
- Annotated cell types based on known markers
- Saved processed data and results for downstream analysis

**Next steps:**
- Differential expression analysis between cell types
- Trajectory analysis for developmental processes
- Integration with other datasets
- Functional enrichment analysis

---